In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch
import torchvision
import torchvision.transforms as transforms

# 테스트 데이터 로드
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # CIFAR-10 정규화
])

test_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False, # 테스트 데이터 로드
    download=True,
    transform=transform
)

test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=64, # 배치 크기
    shuffle=False, # 순차적으로 데이터 로드
    num_workers=0  # ★ 변경: 2 -> 0 Windows에서 __main__ 가드 없이도 에러 안 나도록
)

class CNN_MLP(nn.Module):
    def __init__(self):
        super(CNN_MLP, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(32 * 14 * 14, 128)  # ★ 변경: 32*8*8 -> 32*14*14 (실제 출력 크기와 일치)
        self.fc2 = nn.Linear(128, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(self.relu(self.conv2(x)))
        x = torch.flatten(x, start_dim=1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# 모델 초기화
model = CNN_MLP()

# 학습 단계
epochs = 10
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()  # ★ 추가: 손실함수 정의

for epoch in range(epochs):  # ★ 추가: epoch 루프
    for i, (images, labels) in enumerate(test_loader): # train_loader -> test_loader (그대로 둠)
        # 순전파 (forward)
        outputs = model(images)
        loss = criterion(outputs, labels)

        # 역전파(backward)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')  # ★ 수정: epoch 변수 사용 가능

def evaluate_model(model, test_loader):
    model.eval() # 평가 모드 (dropout, batchnorm 비활성화)
    correct = 0 # 맞춘 샘플 개수
    total = 0 # 전체 샘플 개수

    with torch.no_grad(): # 평가 중에는 기울기 계산하지 않음
        for inputs, labels in test_loader:
            outputs = model(inputs) # 모델 예측
            _, predicted = torch.max(outputs.data, 1) # 가장 높은 점수의 클래스 선택
            total += labels.size(0) # 전체 샘플 개수 업데이트
            correct += (predicted == labels).sum().item() # 맞춘 샘플 개수 업데이트

    accuracy = correct / total * 100 # 정확도 계산 (백분율)
    print(f'테스트 데이터셋 정확도: {accuracy:.2f}%')

# 모델 평가
evaluate_model(model, test_loader)

Epoch [1/10], Loss: 1.5618
Epoch [2/10], Loss: 1.0140
Epoch [3/10], Loss: 0.6139
Epoch [4/10], Loss: 0.3046
Epoch [5/10], Loss: 0.1368
Epoch [6/10], Loss: 0.1215
Epoch [7/10], Loss: 0.0399
Epoch [8/10], Loss: 0.1147
Epoch [9/10], Loss: 0.9776
Epoch [10/10], Loss: 0.0231
테스트 데이터셋 정확도: 91.62%
